In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
# pytorch提供的模块
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.2001,  0.2031,  0.0702, -0.2050,  0.3053, -0.0404,  0.0411,  0.0603,
         -0.0283,  0.1784],
        [-0.2428,  0.1747,  0.0389, -0.1910,  0.2609, -0.0039, -0.0280,  0.0703,
         -0.0502,  0.1538]], grad_fn=<AddmmBackward0>)

In [3]:
# 自定义块
"""
一个块需要的基本功能：
1. 将输⼊数据作为其前向传播函数的参数。
2. 通过前向传播函数来⽣成输出。forward函数
3. 计算其输出关于输⼊的梯度，可通过其反向传播函数进⾏访问。通常这是⾃动发⽣的。backward函数
4. 存储和访问前向传播计算所需的参数。
5. 根据需要初始化模型参数。
"""
"""
继承nn.Module的类实现块，只需要实现__init__和前向传播函数
"""


class MLP(nn.Module):
    def __init__(self):
        # 调⽤MLP的⽗类Module的构造函数来执⾏必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        # 定义隐藏层
        self.hidden = nn.Linear(20, 256)
        # 定义输出层
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


net = MLP()
net(X)

tensor([[-0.0930, -0.2091,  0.1396,  0.1756, -0.1634,  0.1720, -0.1835,  0.2592,
          0.0378, -0.1421],
        [-0.0449, -0.1525,  0.1013,  0.0653, -0.1765,  0.1730, -0.1844,  0.1758,
          0.0391, -0.1253]], grad_fn=<AddmmBackward0>)

In [4]:
# 顺序块，实现Sequential
"""
Sequential需要拥有的功能
1. ⼀种将块逐个追加到列表中的函数；
2. ⼀种前向传播函数，⽤于将输⼊按追加块的顺序传递给块组成的“链条”。
"""


class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            print(idx, module)
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

0 Linear(in_features=20, out_features=256, bias=True)
1 ReLU()
2 Linear(in_features=256, out_features=10, bias=True)


tensor([[ 0.0239, -0.0984, -0.2156,  0.0136,  0.1265, -0.1341, -0.0572, -0.1356,
         -0.2161,  0.2588],
        [-0.0683, -0.1639, -0.1130, -0.1049,  0.1173, -0.0862, -0.0838, -0.1253,
         -0.1882,  0.1580]], grad_fn=<AddmmBackward0>)

In [6]:
# 前向传播中执行自定义代码
"""
⽹络中的所有操作都对⽹络的激活值及⽹络的参数起作⽤。
有时可能希望合并既不是上⼀层的结果也不是可更新参数的项，称之为常数参数。
例如，需要⼀个计算函数 f(x; w) = cWX的层，其中x是输⼊， w是参数， c是某个在优化过程中没有更新的指定常量。
"""


class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        # 进行wx
        X = self.linear(X)
        # 使⽤创建的常量参数以及relu和mm函数
        # 进行cwx+1
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复⽤全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        # 在L1范数⼤于1的条件下，将输出向量除以2，直到它满⾜条件为⽌
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


net = FixedHiddenMLP()
net(X)

tensor(0.2956, grad_fn=<SumBackward0>)

In [8]:
# 组合块,可以混合搭配各种组合块的⽅法。
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))


chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.5037, grad_fn=<SumBackward0>)